In [27]:
#Step 0a: Bringing the necessary packages in
from imdb import IMDb
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import cPickle as pickle
ia = IMDb(accessSystem='http')
from collections import defaultdict 
import io

In [28]:
from imdb import IMDb 
from bs4 import BeautifulSoup
import cPickle as pickle 
import io
import time
import requests

In [29]:
pages_by_year = {}

pages = xrange(1,9)
pagestext = {}
for page in pages: 
    r = requests.get("http://www.boxofficemojo.com/yearly/chart/?page=%s&view=releasedate&view2=domestic&yr=2014&p=.htm"%page)
    pagestext[page] = r.text
    time.sleep(1)

In [30]:
from collections import defaultdict 
movie_budget = defaultdict(list) 
year = 2014
for page in pages: 
    soup = BeautifulSoup(pagestext[page], "html.parser")
    rows = soup.find_all("font", attrs={'size':'2'})
           
    start = 10 
    for i in range(start,len(rows)-2):
        t = rows[i].get_text()
        if unicode('Summary of') in t: 
            break
        elif (i-start) % 9 == 0: 
            movie_budget['rank'].append(t)
        elif (i-start) % 9 == 1: 
            r = '('+str(year)+')'
            if unicode(r) in t: 
                j = t.index(unicode(r))
                movie_budget['title'].append(t[:j])
            else: 
                movie_budget['title'].append(t)
        elif (i-start) % 9 == 2: 
            movie_budget['studio'].append(t)
        elif (i-start) % 9 == 3: 
            movie_budget['gross'].append(t)
        elif (i-start) % 9 == 4: 
            movie_budget['gross theaters'].append(t)
        elif (i-start) % 9 == 5: 
            movie_budget['opening'].append(t)
        elif (i-start) % 9 == 6: 
            movie_budget['opening theaters'].append(t)
        elif (i-start) % 9 == 7: 
            movie_budget['open'].append(t)
        elif (i-start) % 9 == 8: 
            movie_budget['close'].append(t)

In [31]:
#Step 0b: Introducing the find_movie function for later use
def find_movie(title, year,  mlist):
    # find movies that came out in the same year                                                                                                                                    
    year_list = []
    for movie in mlist:
        try:
            if movie.data['year'] == int(year):
                year_list.append(movie)
        except:
            pass
    ## ADDITION TO DEBORAH'S FUNCTION
    # if only one movie left, return
    if len(year_list) == 1:
        return year_list[0]
    # if the years do not match, there is no match                                                                                                                                  
    if len(year_list) < 1:
        return None
    else:
        # process the title                                                                                                                                                         
        sorted_title = "".join(sorted(title)).replace(" ", "")
        len_sorted_title = len(sorted_title)
        # check whether movies that came out in the same year                                                                                                                       
        # have the same letters                                                                                                                                                     
        counts = [0]*len(year_list)
        for j in range(len(year_list)):
            mtitle = year_list[j]['title']
            sorted_mtitle = "".join(sorted(mtitle)).replace(" ", "")
            if len_sorted_title == len(sorted_mtitle):
                # if the title cannot be converted to a string                                                                                                                      
                # it is not the correct title                                                                                                                                       
                try:
                    sorted_mtitle = str(sorted_mtitle)
                except:
                    continue
                for i in range(len_sorted_title):
                    if sorted_title[i] == sorted_mtitle[i]:
                        counts[j] += 1
            else:
                continue 
        k = counts.index(max(counts))
        if len(year_list) >= 1:
            return year_list[k]

In [45]:
movie_budget

defaultdict(<type 'list'>, {'gross': [u'$350,126,372', u'$337,135,885', u'$333,176,600', u'$259,766,572', u'$257,760,692', u'$255,119,788', u'$245,439,076', u'$241,410,378', u'$233,921,534', u'$222,527,828', u'$208,545,589', u'$202,853,933', u'$200,676,069', u'$191,719,337', u'$191,204,754', u'$188,020,017', u'$177,002,924', u'$167,767,189', u'$150,947,895', u'$150,157,400', u'$134,938,200', u'$131,538,435', u'$128,002,372', u'$126,663,600', u'$124,872,350', u'$115,637,895', u'$113,746,621', u'$111,506,430', u'$106,580,051', u'$102,427,862', u'$101,530,738', u'$101,200,044', u'$100,206,256', u'$92,168,600', u'$91,443,253', u'$91,125,683', u'$86,208,010', u'$85,911,262', u'$85,817,906', u'$84,525,432', u'$84,273,813', u'$83,911,193', u'$83,350,911', u'$82,390,774', u'$78,031,620', u'$72,688,614', u'$71,962,800', u'$66,954,149', u'$65,182,182', u'$65,014,513', u'$64,251,541', u'$60,755,732', u'$59,700,064', u'$59,165,787', u'$59,100,318', u'$58,607,007', u'$56,280,355', u'$54,445,357', u

In [9]:
ia.search_movie('Guardians of the Galaxy')

[<Movie id:2015381[http] title:_Guardians of the Galaxy (2014)_>,
 <Movie id:4144428[http] title:_"Behind the Scenes (I)" Guardians of the Galaxy (I) (2014)_>,
 <Movie id:4384952[http] title:_"Sit Down with the Stars" Guardians of the Galaxy (2014)_>,
 <Movie id:3896198[http] title:_Guardians of the Galaxy Vol. 2 (2017)_>,
 <Movie id:4176370[http] title:_"Marvel's Guardians of the Galaxy" (2015)_>,
 <Movie id:2983766[http] title:_"Ultimate Spider-Man" Guardians of the Galaxy (2013)_>,
 <Movie id:4359280[http] title:_"Someone Has to Review It!" Guardians of the Galaxy (2014)_>,
 <Movie id:3910408[http] title:_"Midnight Screenings" Guardians of the Galaxy (2014)_>,
 <Movie id:4057660[http] title:_"Marvel Disk Wars: The Avengers" Guardians of the Galaxy! (2014)_>,
 <Movie id:4266858[http] title:_"Honest Trailers" Guardians of the Galaxy (2014)_>,
 <Movie id:4665948[http] title:_"Guardians of the Galaxy (2015) (TV Episode) - Jam Bites" (2014) (mini)_>,
 <Movie id:3951492[http] title:_"Unsc

In [7]:
a = find_movie(movie_budget['title'][12], 2014, ia.search_movie(movie_budget['title'][12]))

[<Movie id:0831387[http] title:_Godzilla (2014)_>, <Movie id:3830314[http] title:_Godzilla (2014) (VG)_>]
Gadilloz


In [44]:
movie_budget.keys()

['gross',
 'opening',
 'opening theaters',
 'title',
 'rank',
 'studio',
 'gross theaters',
 'close',
 'open']

In [9]:
ia.get_movie(movie_budget['title'][5])

[<Movie id:2310332[http] title:_The Hobbit: The Battle of the Five Armies (2014)_>,
 <Movie id:4302156[http] title:_"Half in the Bag" The Hobbit: The Battle of the Five Armies (2014)_>,
 <Movie id:5169330[http] title:_"Talking Pictures" The Hobbit: The Battle of the Five Armies (2014)_>,
 <Movie id:4318466[http] title:_"Did You Watch...?" The Hobbit: The Battle of the Five Armies (2014)_>,
 <Movie id:4288952[http] title:_"Midnight Screenings" The Hobbit: The Battle of the Five Armies (2014)_>,
 <Movie id:4058932[http] title:_"The Hobbit: The Battle of Five Armies (2014) (TV Episode) - Hollywood Agent Clark" (2014) (mini)_>,
 <Movie id:4286770[http] title:_"Projector" The Hobbit: The Battle of the Five Armies/Black Sea (2014)_>,
 <Movie id:4328158[http] title:_"Council of Geeks" Battle of the Five Armies - Now Nearly Hobbit Free (2015)_>,
 <Movie id:4561226[http] title:_The Hobbit: The Battle of Five Armies - New Zealand: Home of Middle-Earth - Part 3 (2015) (V)_>,
 <Movie id:4878490[ht

In [33]:
#The next process actually attempts to match the movies in the Box Office dataframe to the 
BOmissingmovies = list() # To hold anything we don't find
BOdict = {} # This will actually contain the movies and information about them.

movienumber = len(movie_budget['gross'])

for i in range(0,movienumber):
    movieobj = None
    # Need to process the row-level information out of BOdf in order to get the movie objects
    movieobj = ia.search_movie(movie_budget['title'][i])
    
    #Handling cases where we haven't found the movie or have multiples
    if movieobj is None or len(movieobj)>1:
        movieobj = find_movie(movie_budget['title'][i], 2014, ia.search_movie(movie_budget['title'][i])) # find the movie
        if type(movieobj) == list: 
            movieobj = movieobj[0]
            
    print (i, movie_budget['title'][i], movieobj)

    if movieobj is not None and not (type(movieobj) == list and len(movieobj) == 0):
        ## Get movie id ##
        if type(movieobj) == list: 
            movieobj = movieobj[0]
        ia.update(movieobj)    
        movid = movieobj.movieID
        ## Populate dictionary, main key is movie id ##
        BOdict[movid] = {}
        # "title": title of movie
        BOdict[movid]['title'] = movie_budget['title'][i]
        # "nominations": list of Oscar nominations
        BOdict[movid]['gross'] = movie_budget['gross'][i]
        # "won": list of Oscars won
        BOdict[movid]['opening'] = movie_budget['opening'][i]
        # "year": year Oscar won
        BOdict[movid]['rank'] = movie_budget['rank'][i]
        # "country": country of movie
        BOdict[movid]['studio'] = movie_budget['studio'][i] 
        BOdict[movid]['open'] = movie_budget['open'][i]
        BOdict[movid]['close'] = movie_budget['close'][i]
        BOdict[movid]['opening theaters'] = movie_budget['opening theaters'][i]
        BOdict[movid]['rating'] = movieobj['rating']
        BOdict[movid]['mpaa'] = movieobj['mpaa']
        BOdict[movid]['director'] = movieobj['director']
        BOdict[movid]['cast'] = movieobj['cast']
        #BOdict[movid]['']
        # make each award individual key and the value to indicate whether movie won/nominated or not
        # Loop through awards list and indicate if movie was nominated or won
    else:
        BOmissingmovies.append((i, movie_budget['title'][i], 2014))
        

(0, u'American Sniper', <Movie id:2179136[http] title:_American Sniper (2014)_>)
(1, u'The Hunger Games: Mockingjay - Part 1', <Movie id:1951265[http] title:_The Hunger Games: Mockingjay - Part 1 (2014)_>)
(2, u'Guardians of the Galaxy', <Movie id:2015381[http] title:_Guardians of the Galaxy (2014)_>)
(3, u'Captain America: The Winter Soldier', <Movie id:1843866[http] title:_Captain America: The Winter Soldier (2014)_>)
(4, u'The LEGO Movie', <Movie id:1490017[http] title:_The Lego Movie (2014)_>)
(5, u'The Hobbit: The Battle of the Five Armies', <Movie id:2310332[http] title:_The Hobbit: The Battle of the Five Armies (2014)_>)
(6, u'Transformers: Age of Extinction', <Movie id:2109248[http] title:_Transformers: Age of Extinction (2014)_>)
(7, u'Maleficent', <Movie id:1587310[http] title:_Maleficent (2014)_>)
(8, u'X-Men: Days of Future Past', <Movie id:1877832[http] title:_X-Men: Days of Future Past (2014)_>)
(9, u'Big Hero 6', <Movie id:2245084[http] title:_Big Hero 6 (2014)_>)
(10, u

In [18]:
a = find_movie(movie_budget['title'][122],2014,ia.search_movie(movie_budget['title'][122]))

[<Movie id:0458481[http] title:_Sin City: A Dame to Kill For (2014)_>]


In [46]:
movie_budget.keys()

['gross',
 'opening',
 'opening theaters',
 'title',
 'rank',
 'studio',
 'gross theaters',
 'close',
 'open']

In [42]:
temp2 = BOdict['2467046']

In [47]:
movieitself = ia.get_movie('2467046')

In [91]:
from bs4 import BeautifulSoup
from collections import defaultdict 
from pyquery import PyQuery as pq
import time
import requests

In [92]:
r_numbers = requests.get("http://www.the-numbers.com/movie/budgets/all")
rn_text = r_numbers.text 

In [93]:
d_=pq(rn_text)
d_tables=pq(d_('table'))
rows = pq(d_tables[0])('tr')

In [94]:
budget = defaultdict(list)
for j in range(1,len(rows)):
    dat = pq(rows[j])('td')
    for i in range(len(dat)): 
        if i % 6 == 1:
            budget['year'].append(int(pq(dat[i])('a').text().split('/')[-1]))
        elif i % 6 == 2:
            budget['title'].append(pq(dat[i])('a').text())
        elif i % 6 == 3: 
            budget['budget'].append(pq(dat[i]).text())

In [95]:
for movie in BOdict.keys():
    t = BOdict[movie]['title']
    try:
        j  = budget['title'].index(t)
        BOdict[movie]['budget'] = budget['budget'][j]
    except:
        continue
    

In [96]:
BOdict['2179136']

{'budget': '$58,000,000',
 'gross': u'$350,126,372',
 'opening': u'$633,456',
 'rank': u'1',
 'studio': u'WB',
 'title': u'American Sniper'}

In [97]:
temp1 = pd.DataFrame(BOdict)
temp1 = temp1.transpose()
# Getting 123 rows with budget having matched to IMDBpy

In [98]:
for k in budget.keys():
    print k
    print len(budget[k])

title
5022
budget
5022
year
5022


In [99]:
budgetdf = pd.DataFrame(budget)

In [104]:
budgetdf[budgetdf['year']==2014]

,budget,title,year
12,"$250,000,000",The Hobbit: The Battle of the Five Armies,2014
24,"$210,000,000",Transformers: Age of Extinction,2014
38,"$200,000,000",X-Men: Days of Future Past,2014
39,"$200,000,000",The Amazing Spider-Man 2,2014
62,"$180,000,000",Maleficent,2014
68,"$178,000,000",Edge of Tomorrow,2014
79,"$170,000,000",Guardians of the Galaxy,2014
81,"$170,000,000",Captain America: The Winter Soldier,2014
82,"$170,000,000",Dawn of the Planet of the Apes,2014
89,"$165,000,000",Big Hero 6,2014


In [105]:
temp1

,budget,gross,opening,rank,rating,studio,title
0086506,NaN,"$13,261","$1,756",547,NaN,Icar.,One Day Pina Asked...
0262792,NaN,"$28,516","$9,423",488,NaN,APD/C,Stations of the Elevated
0365907,"$28,000,000","$26,307,600","$12,758,780",98,NaN,Uni.,A Walk Among the Tombstones
0435651,"$25,000,000","$45,090,374","$12,305,016",73,NaN,Wein.,The Giver
0455944,"$55,000,000","$101,530,738","$34,137,828",31,NaN,Sony,The Equalizer
0458413,NaN,"$13,347","$1,426",545,NaN,Magn.,A Long Way Down
0458481,NaN,"$13,757,804","$6,317,683",123,NaN,W/Dim.,Frank Miller's Sin City: A Dame to Kill For
0469021,NaN,"$153,426","$12,664",337,NaN,Magn.,Alan Partridge: The Movie
0498351,NaN,"$32,188","$24,260",475,NaN,Magn.,The Hero of Color City
0780548,NaN,"$5,565","$5,565",634,NaN,WGUSA,Kid Cannabis
